In [1]:
import pandas as pd
from requests import get
from bs4 import BeautifulSoup
import numpy as np

In [2]:
response = get('https://nturanking.csti.tw/ranking/ByCountry/2024/US')

In [3]:
html_soup = BeautifulSoup(response.text,'html')
html_soup

<!DOCTYPE html>
<html lang="tw">
<!-- 網頁鎖右鍵 -->
<!-- <div onselectstart="return false;" ondragstart="return false;" oncontextmenu="return false;" oncopy="return false;"> -->
<head>
<meta charset="utf-8"/>
<link href="/static/NTURankingData/img/favicon.ico" rel="shortcut icon"/>
<!-- Google tag (gtag.js) -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-4LNKEDXF3M"></script>
<script>
    window.dataLayer = window.dataLayer || [];
    function gtag(){dataLayer.push(arguments);}
    gtag('js', new Date());

    gtag('config', 'G-4LNKEDXF3M');
    </script>
<link href="/static/assets/tether/tether.min.css" rel="stylesheet" type="text/css"/>
<link href="/static/assets/bootstrap/css/bootstrap.min.css" rel="stylesheet" type="text/css"/>
<link href="/static/assets/bootstrap/css/bootstrap-grid.min.css" rel="stylesheet" type="text/css"/>
<link href="/static/assets/bootstrap/css/bootstrap-reboot.min.css" rel="stylesheet" type="text/css"/>
<link href="/static/assets/dropdown/

In [4]:
research_rankings = html_soup.find_all('td')

rank_list = [str(research_rankings[i:i + 12]) for i in range(0, len(research_rankings), 12)]

uni_names = [rank_list[i].split('</a>')[0].split('>')[-1].replace(", ", "-") 
             for i in range(0, len(rank_list))]

uni_scores = [float(rank_list[i].split('<td>')[3].split('</td>')[0]) 
             if len(rank_list[i].split('<td>')[3].split('</td>')[0]) == 4 
             else np.nan 
             for i in range(0, len(rank_list))]

In [5]:
universities = ["Princeton University",
                "Massachusetts Institute of Technology",
                "Harvard University",
                "Stanford University",
                "Yale University",
                "California Institute of Technology",
                "Duke University",
                "Johns Hopkins University",
                "Northwestern University",
                "University of Pennsylvania",
                "Cornell University",
                "University of Chicago",
                "Brown University",
                "Columbia University",
                "Dartmouth College",
                "University of California-Los Angeles",
                "University of California-Berkeley",
                "Rice University",
                "University of Notre Dame",
                "Vanderbilt University",
                "Carnegie Mellon University",
                "University of Michigan-Ann Arbor",
                "Washington University in St. Louis",
                "Emory University",
                "Georgetown University",
                "University of Virginia",
                "University of North Carolina-Chapel Hill",
                "University of Southern California",
                "University of California-San Diego",
                "New York University",
                "University of Florida",
                "University of Texas-Austin",
                "Georgia Institute of Technology",
                "University of California-Davis",
                "University of California-Irvine",
                "University of Illinois-Urbana-Champaign",
                "Boston College",
                "Tufts University",
                "University of California-Santa Barbara",
                "University of Wisconsin-Madison",
                "Boston University",
                "Ohio State University-Columbus",
                "Rutgers University-New Brunswick",
                "University of Maryland-College Park",
                "University of Rochester",
                "Lehigh University",
                "Purdue University",
                "University of Georgia",
                "University of Washington",
                "Wake Forest University"]

In [6]:
missing_unis = [uni for uni in universities if uni not in uni_names]

missing_unis

['University of California-Los Angeles',
 'Purdue University',
 'University of Washington']

In [7]:
uni_names[uni_names.index("University of California- Los Angeles")] = "University of California-Los Angeles"
uni_names[uni_names.index("Purdue University-West Lafayette")] = "Purdue University"
uni_names[uni_names.index("University of Washington-Seattle")] = "University of Washington"

In [8]:
scores_df = pd.DataFrame({"University Name": uni_names,
                          "Research Score": uni_scores})

scores_df.head()

,University Name,Research Score
0,Harvard University,97.6
1,Stanford University,61.5
2,Johns Hopkins University,56.7
3,Massachusetts Institute of Technology,54.9
4,University of Washington,54.5


In [9]:
universities_df = pd.DataFrame(index=universities)

research_df = universities_df.merge(right=scores_df, how='left', left_index=True, right_on='University Name')

In [10]:
research_df = research_df.sort_values('Research Score', ascending=False).reset_index(drop=True)
research_df = research_df.fillna('30')

In [11]:
research_df.to_csv("ntu_research_ranking.csv", index=False)